<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук ВШЭ, программист-исследователь Mail.ru Group  Юрий Кашницкий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 4. Сравнение алгоритмов классификации на наборах данных репозитория UCI</center>

В рамках данного задания нужно выбрать один из наборов данных для задачи классификации из [репозитория UCI](http://archive.ics.uci.edu/ml/), выбрать несколько алгоритмов классификации и сравнить их на этом наборе данных.

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%pylab inline
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


**Выберите любой набор данных для задачи классификации из [репозитория UCI](http://archive.ics.uci.edu/ml/) и загрузите его, используя функцию urlopen. Конечно, можно и скачать данные, а потом открыть csv-файл с помощью pandas.**

In [3]:
import sys
if sys.version_info.major == 2:
    from urllib import urlopen
elif sys.version_info.major == 3:
    from urllib.request import urlopen

url = # Ваш код здесь
raw_data = urlopen(url)
data = np.loadtxt(raw_data, delimiter=",")

SyntaxError: invalid syntax (<ipython-input-3-b6abf4e9aadb>, line 7)

In [28]:
# либо
import pandas as pd
data = pd.read_csv('../../data/adult.data.csv')

**Выведите основные характеристики выборки (число объектов, признаков, число классов) и разделите выборку на обучение и контроль с помощью функции train_test_split**

In [52]:
#from sklearn.cross_validation import train_test_split # for sklearn < 18.0
from sklearn.model_selection import train_test_split, KFold

# Print basic data stats: 
# number of objects, number of features, number of classes 
print(data.shape)

(32561, 15)


In [32]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [85]:
fact_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'salary']
for cols in fact_cols:
    data[cols] = pd.factorize(data[cols])[0]
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [86]:
# Extract objects and labels from data
y = data['salary']
X = data.drop('salary', 1)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

## 1. Классификаторы без настройки параметров 

**Выберите несколько (2-3) классификаторов из sklearn. Для удобства (чтобы не переписывать один и тот же код для каждого классификатора) можно создать список классификаторов, который, например, может состоять из пар вида (classifier_instance(), 'classifier name').**

In [88]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
names = ['kNN', 'logit']
classifiers = [KNeighborsClassifier(), LogisticRegression()]

**С помощью 5-кратной кросс-валидации оцените качество каждого классификатора на обучающей выборке. Используйте функцию cross_val_score. Попробуйте использовать разные метрики качества (параметр scoring)**

In [103]:
# from sklearn.cross_validation import cross_val_score # for sklearn < 18.0
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5)
scores = []
scoring_params = ['accuracy', 'roc_auc', 'precision', 'recall', 'neg_log_loss']

# Do it for each classifier
for name, clf in list(zip(names, classifiers)):
    for param in scoring_params:
        clf_scores = cross_val_score(clf, X_train, y_train, cv=KFold(n_folds), scoring=param)
        clf_mean_score = np.mean(clf_scores)
        scores.append((name, param, clf_mean_score))

# Print the name of classifier with
# the best score and the score
print(scores)
print(max(scores,key=lambda x:x[2]))

[('kNN', 'accuracy', 0.77408770831767304), ('kNN', 'roc_auc', 0.65833297400270385), ('kNN', 'precision', 0.55262384053604885), ('kNN', 'recall', 0.30122050719650606), ('kNN', 'neg_log_loss', -2.1957945147828637), ('logit', 'accuracy', 0.79993021148010091), ('logit', 'roc_auc', 0.60154839248600955), ('logit', 'precision', 0.72702814510227887), ('logit', 'recall', 0.26438943924257419), ('logit', 'neg_log_loss', -0.51053574061876517)]
('logit', 'accuracy', 0.79993021148010091)


Логистическая регрессия показывает лучший результат при всех вариантах параметров оценки.

**Обучите те же классификаторы без параметров на всей обучающей выборке и проверьте их качество на тестовой выборке. Является ли лучшим классификатор, который имеет наибольшее качество на кросс-валидации?**

In [ ]:
from sklearn.metrics import accuracy_score

scores = []

# Do it for each classifier
clf = # Ваш код здесь
clf.fit # Ваш код здесь
y_pred = clf.predict # Ваш код здесь
clf_score = accuracy_score # Ваш код здесь
scores.append(clf_score)

# Print the name of classifier with
# the best score and the score
print # Ваш код здесь

## 2. Классификаторы с настройкой параметров 

**Настройте параметры выбранных вами алгоритмов с помощью GridSearchCV и выберите лучший классификатор**

In [ ]:
from sklearn.grid_search import GridSearchCV # for sklearn < 18.0
from sklearn.model_selection import GridSearchCV 

n_folds = 5
scores = []

# Do it for each classifier
clf_params = # Ваш код здесь
grid = GridSearchCV # Ваш код здесь
clf_best_score = # Ваш код здесь
scores.append(clf_best_score)

# Print the name of classifier with
# the best score, the score and the params
print # Ваш код здесь

**Для лучшего классификатора и набора параметров для него выберите более мелкую сетку (для того чтобы уточнить лучшие значения параметров). Для этого классификатора настройте параметры по обучающей выборке с помощью GridSearchCV. Постройте график зависимости среднего качества на кросс-валидации от значений параметров**

In [ ]:
clf_params = # Ваш код здесь
grid = GridSearchCV # Ваш код здесь
clf_best_score = # Ваш код здесь
clf_best_params = # Ваш код здесь
clf_best = # Ваш код здесь

mean_validation_scores = # Ваш код здесь
plt.plot # Ваш код здесь

# Print the best parameters and score
print # Ваш код здесь

**Для полученной модели посчитайте качество на тестовой выборке и сравните его с качеством, полученным без настройки параметров**

In [ ]:
y_pred = clf_best.predict # Ваш код здесь
clf_score = accuracy_score # Ваш код здесь

# Print the score
print # Ваш код здесь

**Можно поварьировать параметр scoring и позапускать написанный код ещё раз. Изменится ли лучший классификатор?**

## Ссылки:

 - <a href="http://archive.ics.uci.edu/ml/">UCI Machine Learning Repository</a> 
 - <a href="http://scikit-learn.org/stable/index.html">Scikit-learn documentation</a>